# Prenight Briefing for {{ params.day_obs }}

This is presently just a playground for building a pre-night briefing.

In [ ]:
# This cell is only for setting parameter defaults
day_obs = "2025-05-05"
sim_date = "2024-01-04"
sim_index = "2"

In [ ]:
# Validate the input
import re
assert re.match(r'^\d\d\d\d-\d\d-\d\d$', day_obs) is not None
assert re.match(r'^\d\d\d\d-\d\d-\d\d$', sim_date) is not None
assert re.match(r'^\d+$', sim_index) is not None

In [ ]:
from IPython.display import display, HTML
import datetime
import sys
import os
import yaml
from urllib.parse import urlparse
import warnings
import pandas as pd
import numpy as np
import astropy
import bokeh
import bokeh.io
import boto3
from astropy.time import Time

In [ ]:
os.environ['RUBIN_SIM_DATA_DIR'] = '/sdf/data/rubin/user/neilsen/data/rubin_sim_data'

In [ ]:
sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/pip_targets/lib/python3.11/site-packages')

In [ ]:
devel_versions = True
if devel_versions:
    sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/times_square_sources/2024-03-25/uranography')
    sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/times_square_sources/2024-03-25/rubin_scheduler')
    sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/times_square_sources/2024-03-25/rubin_sim')
    sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/times_square_sources/2024-03-25/schedview')

In [ ]:
import rubin_scheduler
import rubin_scheduler.utils
from rubin_scheduler.scheduler.model_observatory import ModelObservatory
from rubin_sim import maf
from lsst.resources import ResourcePath

In [ ]:
import schedview.compute
import schedview.compute.visits
import schedview.collect
import schedview.plot

In [ ]:
astropy.utils.iers.conf.iers_degraded_accuracy = 'ignore'

In [ ]:
# Degraded IERS accuracy is never going to be important for these figures.

# If IERS degraded accuracy encountered, don't fail, just keep going.
astropy.utils.iers.conf.iers_degraded_accuracy = "ignore"

# Don't even complain.
warnings.filterwarnings(
    "ignore",
    category=astropy.utils.exceptions.AstropyWarning,
    message="Tried to get polar motions for times after IERS data is valid. Defaulting to polar motion from the 50-yr mean for those. This may affect precision at the arcsec level. Please check your astropy.utils.iers.conf.iers_auto_url and point it to a newer version if necessary.",
)

In [ ]:
# Don't complain about working with daytime MJDs either.
warnings.filterwarnings(
    "ignore",
    module="rubin_scheduler.skybrightness_pre.sky_model_pre",
    category=UserWarning,
    message="Requested MJD between sunrise and sunset, returning closest maps",
)

In [ ]:
# Quiet unimportant chatter from healpy.
healpy_logger = logging.getLogger("healpy")
healpy_logger.setLevel(logging.WARNING)

In [ ]:
bokeh.io.output_notebook()

In [ ]:
%matplotlib inline

In [ ]:
archive_uri = "s3://rubin-scheduler-prenight/opsim/"
# archive_uri = "file:///sdf/data/rubin/user/neilsen/data/test_sim_archive/"

if urlparse(archive_uri).scheme.upper() == 'S3':
    os.environ["S3_ENDPOINT_URL"] = "https://s3dfrgw.slac.stanford.edu/"
    available_profiles = boto3.session.Session().available_profiles
    for candidate_profile in ('prenight', 'default'):
        if candidate_profile in available_profiles:
            os.environ["AWS_PROFILE"] = candidate_profile
            break

In [ ]:
sim_archive_rp = ResourcePath(archive_uri).join(sim_date, forceDirectory=True).join(sim_index, forceDirectory=True)
sim_archive_metadata = yaml.safe_load(sim_archive_rp.join('sim_metadata.yaml').read().decode())

In [ ]:
sim_rp = sim_archive_rp.join(sim_archive_metadata['files']['observations']['name'])
day_obs_mjd = int(Time(day_obs).mjd)
observatory = ModelObservatory(init_load_length=1)
timezone = "Chile/Continental"

## Astronomical events during the night

In [ ]:
day_obs_datetime = Time(day_obs_mjd, format='mjd').datetime
day_obs_date = datetime.date(day_obs_datetime.year, day_obs_datetime.month, day_obs_datetime.day)
night_events = schedview.compute.astro.night_events(day_obs_date)
night_events

## Sun and moon positions in the middle of the night

In [ ]:
model_observatory = ModelObservatory(init_load_length=1)
model_observatory.mjd = night_events.loc['night_middle', 'MJD']

In [ ]:
body_positions_wide = pd.DataFrame(model_observatory.almanac.get_sun_moon_positions(night_events.loc['night_middle', 'MJD']))
body_positions_wide.index.name = 'r'
body_positions_wide.reset_index(inplace=True)

angle_columns = ['RA', 'dec', 'alt', 'az']
all_columns = angle_columns + ['phase']
body_positions = (
    pd.wide_to_long(body_positions_wide, stubnames=('sun', 'moon'), suffix=r'.*', sep='_', i='r', j='')
    .droplevel('r')
    .T[all_columns]
)
body_positions[angle_columns] = np.degrees(body_positions[angle_columns])
body_positions

All angles are in degrees.

## Simulated visits

In [ ]:
try:
    visits = schedview.collect.read_opsim(
        sim_rp,
        constraint=f"FLOOR(observationStartMJD-0.5)={day_obs_mjd}",
        stackers=[
            maf.stackers.TeffStacker(),
            maf.stackers.ObservationStartDatetime64Stacker(),
            maf.stackers.DayObsStacker(),
            maf.stackers.DayObsMJDStacker(),
            maf.stackers.DayObsISOStacker(),
            maf.stackers.OverheadStacker()
        ]        
    )
    visits = schedview.compute.visits.add_coords_tuple(visits)
except UserError:
    visits = pd.DataFrame(rubin_scheduler.scheduler.utils.empty_observation()).drop(index=0)

### Numbers of exposures, and gaps between them

In [ ]:
if len(visits):
    overhead_summary = schedview.compute.visits.compute_overhead_summary(visits, night_events.loc['sun_n12_setting','MJD'], night_events.loc['sun_n12_rising','MJD'])
    summary_table = schedview.plot.create_overhead_summary_table(overhead_summary)
    display(HTML(summary_table))
else:
    print("No visits")

### Map of the visits

In [ ]:
if len(visits):
    vmap, vmap_data = schedview.plot.visitmap.create_visit_skymaps(
        visits=visits,
        night_date=day_obs_date,
        timezone=timezone,
        observatory=observatory,
    )
    bokeh.io.show(vmap)
else:
    print("No visits")

The above plots show the visits collected during the night in two different representations, modeled after physical observing tools.

 - The "Armillary sphere" shows the sphere in orthographic projection, with the center point of the projection controlled by the "center alt" and "center az" sliders beneath the plot. A static orthogrophic projection is not an equal-area projection, but playing with the sliders is a helpful way to inform a human's spatial reasoning in three dimensions.
 - The "Planisphere" shows the sky in [Lambert Azimuthal Equal Area Projection](https://en.wikipedia.org/wiki/Lambert_azimuthal_equal-area_projection), centered at the south celestial pole, with R.A. increasing counterclockwise. The projection used is equal area, but highly distorted near the north celestial pole (outside the LSST footprint). This is a particularly helpful representation for planning observing, because changes in time in relevant features are simple rotations, without alterations in distortion, and there are no discontinuities anywhere in the footprint at any time of year.

Both plots show the footprints of camera pointing taken up to the time set by the MJD slider, with the most recent three pointings outlined in cyan. The fill colors are set according to the [descolors palette](https://github.com/DarkEnergySurvey/descolors):

 - <span style='background-color:#56b4e9'>&nbsp;&nbsp;&nbsp;</span><span style='color:#56b4e9'> blue</span>: u band
 - <span style='background-color:#008060'>&nbsp;&nbsp;&nbsp;</span><span style='color:#008060'> green</span>: g band
 - <span style='background-color:#ff4000'>&nbsp;&nbsp;&nbsp;</span><span style='color:#ff4000'> red</span>: r band
 - <span style='background-color:#850000'>&nbsp;&nbsp;&nbsp;</span><span style='color:#850000'> brown/crimson</span>: i band
 - <span style='background-color:#6600cc'>&nbsp;&nbsp;&nbsp;</span><span style='color:#6600cc'> purple</span>: z band
 - <span style='background-color:#000000'>&nbsp;&nbsp;&nbsp;</span><span style='color:#000000'> black</span>: y band

Both plots have the following additional annotations:
 - The gray background shows the planned final depth of the LSST survey.
 - The orange disk shows the coordinates of the moon.
 - The yellow disk shows the coordinates of the sun.
 - The green line (oval) shows the ecliptic.
   - The sun moves along the ecliptic in the direction of increasing R.A. (counter-clockwise in the planisphere figure) such that it makes a full revolution in one year.
   - The moon moves roughly (within 5.14°) along the ecliptic in the direction of increasing R.A. (counter-clockwise in the planisphere figure) , completing a full revolution in one [sidereal month](https://en.wikipedia.org/wiki/Lunar_month#Sidereal_month) (a bit over 27 days), about 14° per day.
 - The blue line (oval) shows the plane of the Milky Way.
 - The black line shows the horizon at the time set by the MJD slider.
 - The red line shows a zenith distince of 70° (airmass=2.9) at the time set by the MJD slider.

### Altitude plot

In [ ]:
if len(visits):
    fig = schedview.plot.nightly.plot_alt_vs_time(
        visits=visits, almanac_events=night_events
    )
    bokeh.io.show(fig)
else:
    print("No visits")

### Horizon plot

In [ ]:
if len(visits):
    fig = schedview.plot.nightly.plot_polar_alt_az(visits=visits, legend=True)
    bokeh.io.show(fig)
else:
    print("No visits")

### Table of visits

In [ ]:
if len(visits):
    displayed_columns = ['start_date', 'fieldRA', 'fieldDec', 'filter', 'visitExposureTime', 'numExposures', 't_eff', 'skyBrightness', 'seeingFwhmEff', 'cloud', 'note']
    displayed_visits_df = visits.loc[:, displayed_columns]
    with pd.option_context('display.max_rows', 2000):
        display(displayed_visits_df)
else:
    print("No visits")